# **Carga**

Una vez que tenemos los datos limpios y organizados, podemos cargarlos en la base de datos.

In [ ]:
import pandas as pd 
# Cargamos los datos
df_ciudades = pd.read_pickle("../data/data_clean/ciudades.pkl")
df_eventos = pd.read_pickle("../data/data_clean/eventos.pkl")
df_hoteles = pd.read_pickle("../data/data_clean/hoteles.pkl")
df_clientes = pd.read_pickle("../data/data_clean/clientes.pkl")
df_reservas = pd.read_pickle("../data/data_clean/reservas.pkl")

In [3]:
# Nos conectamos a la base de datos que hemos creado y llamado "proyecto_ETL"
import psycopg2
# Importamos os y dotenv para poder recuperar variables del archivo .env
import os
from dotenv import load_dotenv
# Cargamos variables de entorno
load_dotenv()
password = os.getenv("password")
# Configurar la conexión
conn = psycopg2.connect(
    dbname = "proyecto_ETL",
    user = "postgres",
    password = password,
    host="localhost",
    port="5432")
# Crear un cursor para ejecutar comandos SQL
cur = conn.cursor()



In [4]:
# Verificar la conexión
cur.execute("SELECT version();")
print(cur.fetchone())

('PostgreSQL 17rc1 on x86_64-windows, compiled by msvc-19.41.34120, 64-bit',)


## Insertamos datos de ciudades

In [4]:
# Insertamos los datos de ciudades
# Convertir DataFrame a lista de tuplas
records = df_ciudades.to_records(index=False)
records_list = list(records)
query = ("""
    INSERT INTO ciudad (nombre_ciudad)
    VALUES (%s)
""")
cur.executemany(query, records_list)
# Guardar los cambios y cerrar conexión
conn.commit()

print("Registros insertados correctamente")

Registros insertados correctamente


In [5]:
cur.execute("SELECT * FROM ciudad;")

In [6]:
# Mostrar los resultados
for row in cur.fetchall():
    print(row)

(1, 'Madrid')


In [7]:
# A partir de la tabla ciudad sacamos un diccionario con nombre_ciudad : id_ciudad
cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad")
dictio_ciudad = dict(cur.fetchall())
dictio_ciudad

{'Madrid': 1}

In [ ]:
#conn.rollback()

## Insertamos datos de clientes

In [ ]:
# Convertir DataFrame a lista de tuplas
records = df_clientes.to_records(index=False)
records_list = list(records)
query = ("""
    INSERT INTO clientes (id_cliente, nombre, apellido, mail)
    VALUES (%s, %s, %s, %s)
""")
cur.executemany(query, records_list)
# Guardar los cambios y cerrar conexión
conn.commit()

print("Registros insertados correctamente")

Registros insertados correctamente


## Insertamos datos de eventos

In [ ]:
# Insertamos los datos de eventos
# Creamos una lista para meter la información
data_insert_eventos = []
# Comprobamos las columnas del df_eventos
df_eventos.columns

Index(['nombre_evento', 'url_evento', 'codigo_postal', 'direccion', 'horario',
       'fecha_inicio', 'fecha_fin', 'organizacion'],
      dtype='object')

In [11]:
# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in df_eventos.iterrows():
    nombre_evento = row["nombre_evento"]
    url_evento = row["url_evento"]
    codigo_postal = row["codigo_postal"]
    direccion = row["direccion"] 
    horario = row["horario"]
    fecha_inicio = row["fecha_inicio"]
    fecha_fin = row["fecha_fin"]
    organizacion = row["organizacion"]
    id_ciudad = dictio_ciudad.get("Madrid")

    # incluimos los valores en la lista
    data_insert_eventos.append((nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad))

In [12]:
data_insert_eventos[1]

('60 Premio Reina Sofía de Pintura y Escultura',
 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=34c5130393e15910VgnVCM2000001f4a900aRCRD',
 '28009',
 'PASEO COLOMBIA 1',
 '',
 datetime.date(2025, 2, 27),
 datetime.date(2025, 3, 23),
 'Centro Cultural Casa de Vacas (Retiro)',
 1)

In [13]:
# Creamos la query de inserción de los datos de eventos
query_eventos = """ 
                        INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario,
                        fecha_inicio, fecha_fin,organizacion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [14]:
# Insertamos los datos en la tabla eventos
cur.executemany(query_eventos, data_insert_eventos)
# Guardar los cambios
conn.commit()
print("Registros insertados correctamente")

InvalidTextRepresentation: la sintaxis de entrada no es válida para tipo integer: «Dirección no disponible»
LINE 4: ...extoid=5b58d89ad2232910VgnVCM2000001f4a900aRCRD', 'Dirección...
                                                             ^


## Insertamos datos de hoteles

In [15]:
# Insertamos los datos de hoteles
# Creamos una lista para meter la información
data_insert_hoteles = []
# Comprobamos las columnas del df_hoteles
df_hoteles.columns

Index(['nombre_hotel', 'id_hotel', 'valoracion', 'ciudad', 'competencia'], dtype='object')

In [16]:
# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in df_hoteles.iterrows():
    nombre_hotel = row["nombre_hotel"]
    competencia = row["competencia"]
    valoracion = row["valoracion"]
    id_ciudad = dictio_ciudad.get("Madrid")

    # incluimos los valores en la lista
    data_insert_hoteles.append((nombre_hotel, competencia, valoracion, id_ciudad))

In [21]:
conn.rollback()

In [22]:
# Creamos la query de inserción de los datos de hoteles
query_hoteles = """ 
                        INSERT INTO hoteles (nombre_hotel, competencia, valoracion, id_ciudad)
                        VALUES (%s, %s, %s, %s)
"""

In [ ]:
# Insertamos los datos en la tabla hoteles
cur.executemany(query_hoteles, data_insert_hoteles)
# Guardar los cambios
conn.commit()
print("Registros insertados correctamente")

Registros insertados correctamente


## Insertamos datos de reservas

In [24]:
# Creamos una lista para meter la información
data_insert_reservas = []
# Comprobamos las columnas del df_reservas
df_reservas.columns

Index(['id_reserva', 'fecha_reserva', 'inicio_estancia', 'final_estancia',
       'precio_noche', 'id_cliente', 'id_hotel'],
      dtype='object')

In [31]:
conn.rollback()

In [28]:
# Sacamos el id_hotel de la tabla hoteles
cur.execute("SELECT nombre_hotel, id_hotel FROM hoteles")
dictio_hoteles = dict(cur.fetchall())
dictio_hoteles

{'Hotel Monte Verde': 1,
 'Hotel Brisas del Mar': 2,
 'Hotel Camino del Sol': 3,
 'Hotel Puerta del Cielo': 4,
 'Hotel Encanto Real': 5,
 'Palacio del Sol': 6,
 'Hotel Jardines del Rey': 7,
 'Hotel Las Estrellas': 8,
 'Gran Hotel Madrid': 9,
 'Hotel Torre Dorada': 10,
 'Hotel Palacio Imperial': 11,
 'Hotel Luz de Madrid': 12,
 'Hotel Los Almendros': 13,
 'Hotel Sol y Luna': 14,
 'Hotel Mirador Real': 15,
 'Hotel Rincón Sereno': 16,
 'Hotel Vista Alegre': 17,
 'Hotel Costa Azul': 18,
 'Hotel Maravilla Real': 19,
 'ibis Styles Madrid Prado': 20,
 'ibis budget Madrid Calle 30': 21,
 'ibis Madrid Centro las Ventas': 22,
 'ibis budget Madrid Centro las Ventas': 23,
 'ibis budget Madrid Vallecas': 24,
 'ibis Madrid Aeropuerto Barajas': 25,
 'ibis Madrid Alcorcon Tresaguas': 26,
 'ibis budget Madrid Aeropuerto': 27,
 'ibis Madrid Alcobendas': 28,
 'ibis budget Madrid Alcorcon Móstoles': 29}

In [32]:
# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in df_reservas.iterrows():
    id_reserva = row["id_reserva"]
    fecha_reserva = row["fecha_reserva"]
    inicio_estancia = row["inicio_estancia"]
    final_estancia = row["final_estancia"]
    precio_noche = row["precio_noche"]
    id_cliente = row["id_cliente"]
    id_hotel = dictio_hoteles.get(row["nombre_hotel"])

    # incluimos los valores en la lista
    data_insert_reservas.append((id_reserva, fecha_reserva, inicio_estancia, final_estancia,
       precio_noche, id_cliente, id_hotel))

In [33]:
# Creamos la query de inserción de los datos de hoteles
query_reservas = """ 
                        INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia,
       precio_noche, id_cliente, id_hotel)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [34]:
# Insertamos los datos en la tabla reservas
cur.executemany(query_reservas, data_insert_reservas)
# Guardar los cambios
conn.commit()
print("Registros insertados correctamente")

Registros insertados correctamente


In [35]:
# Cerramos el cursor y la conexión
cur.close()
conn.close()